!*********************************************************************!

!

!   This program calculates the unpolarized absorption spectrum and 

!   band dispersion of a linear, 1D aggregate using the Frenkel/

!   charge-transfer Holstein model. See the README file for more 

!   information.

!                                                   Written By:

!                                                   Nicholas Hestand

!*********************************************************************!

In [1]:
import sys
import re
from os import path
import numpy as np

# read in parameters
!*********************************************************************!

!   subroutine reads parameters from user input file 

!   most variables reside in the commonvar module and are used

!   throughout the program in various subroutines

!*********************************************************************!

In [18]:
def read_in_para():
#subroutine read_in_para()

    #logical         exists
    #character*100   buff, label, fname
    #integer         fno, ios, line, pos, errstat
    #parameter       (fno = 201)
    
    #get the input file name as the first argument from command line
        #fname=sys.argv[1]
        fname='/Users/livi/Git/exciton1d-1/PYexample.inp'
    except ValueError:
    #if given otherwise use default parameters    
        if ValueError !=0:
            print('No control file given. Using default parameters')
            c1010
    #check that the given input file exists, abort if not
        if not path.exists(fname):
            print('Input file not found...aborting')
            return
    #open the user input file and read in the parameters
    with open(fname,'r',encoding='utf-8') as f:
        #ios = 0  #the in/out status
        #line = 0 #the current line number
        print('Reading the input file...'+'\n'+'**********************************'+'\n'+'**********************************')
        parameters=['task_title','nmol','vibmax','hw','lambda_n','lambda_c','lambda_a','JCoul','ES1','te','th','ECT','ECTInf','one_state','two_state','ct_state','abs_lw','esnum']
        parameters={i:None for i in parameters}
        for buff in f.readlines():#!continue the loop until end of file,!read a line
            if buff.startswith('#'): #treat as a comment
                continue
            else:#find the label and assign the appropriate value to the variable
                label_buff=buff.split()
                if label_buff[1].isnumeric():
                    label_buff[1]=float(label_buff[1])
                elif label_buff[1] in ['True', 'False']:
                    label_buff[1]=bool(label_buff[1])
                label,buff= label_buff[0],label_buff[1]#parse the line into a label and parameter
                try:
                    parameters[label]=buff
                except:
                    input('invalid label at line','\n',label,'\n','press enter to continue or ctrl+c to abort')
    #print(parameters)
    print('**********************************'+'\n'+'**********************************')
    return parameters
    
def c1010():#not very sure about the function here
    print('Calculating derived parameters in units of hw.')

    #normalize parameters to units of hw    
    parameters['JCoul']  = parameters['JCoul'] /  parameters['hw']
    parameters['ES1']    = parameters['ES1'] /    parameters['hw']
    parameters['te']     = parameters['te'] /     parameters['hw']
    parameters['th']     = parameters['th'] /     parameters['hw']
    parameters['ECT']    = parameters['ECT'] /    parameters['hw']
    parameters['ECTInf'] = parameters['ECTInf'] / parameters['hw']
    parameters['abs_lw'] = parameters['abs_lw'] / parameters['hw']
    
    #Set all Huang-Rhys factors to zero if vibmax is zero
    #This assumes that the user just wants to calculate the
    #free exciton properties
    if ( parameters['vibmax'] == 0 ):
        parameters['lambda_n'] = 0
        parameters['lambda_c'] = 0
        parameters['lambda_a'] = 0
        print('(a)', '>> vibmax is zero. Setting all lambda to zero')

    #set the maximum left and right displacement from a given molecule given periodic boundary conditions
    parameters['nlbnd']  =  -parameters['nmol']/2+(1-1*np.mod(parameters['nmol'],2))
    parameters['nubnd'] =   parameters['nmol']/2
    return parameters


# number of particles


In [ ]:
'''!*********************************************************************!
!                    index the 1 p k states                           !
!A 1-particle k-state is defined in terms of 1-particle local states  !
!as:                                                                  !
!                                                                     !
!   |k,v> = SUM_n exp(i*2*pi*k*n/N)|n,v> / sqrt(N)                    !
!                                                                     !
!   where the vibrations v are in the shifted potential well,         !  
!   molecule n is excited and all other molecules are in their        !
!   ground electronic and vibrational state                           !
!   In this program, we only work with one k-submatrix at a time      !
!   so only the number of vibratons needs to be indexed               !
!*********************************************************************!'''
def index_1p(kount):
    #integer vib
    #allocate the indexing array and initialize as empty
    #index the 1-particle basis states
    #|k,vib> -> nx_1p( vib )
    nx_1p = range(1,vibmax+1)
    
'''!*********************************************************************!
!                  index the 2-particke k states                      !
!A 2-particke k-state is defined in terms of the local states as      !
!                                                                     !
!    |k,v;s,v'> = SUM_n exp(i*2*pi*k*n/N)|n,v;n+s,v'>                 !
!                                                                     !
!where v is the number of vibrations in the shifted potential well    !
!of excited molecule n, and v' is the number of vibrations in the     !
!unshifted well of molecule n+s. All other molecules are in their     !
!ground electronic and vibrational states.                            !
!*********************************************************************!'''
def index_2p(kount)
    #integer vib, s, svib
    #allocate the indexing array and set as empty
    nx_2p=np.zeros(vibmax,nubnd-nlbnd,vibmax-1)    
    #index the two-particle basis states
    #|k,vib;s,svib> -> nx_2p( vib, s, svib )
    for vib in range(0,vibmax):#vibration on electronexcited molecule
        for s in range(nlbnd,nubnd):#displacement from electronic excited
            if s==0:#displacement cannot be zero
                break
            for svib in range(1, vibmax):#vibration on ground state molecule
                if ( vib + svib > vibmax ):#truncate at vibmax
                    break  #truncate at vibmax
                kount = kount + 1
                nx_2p( vib, s, svib ) = kount

'''!*********************************************************************!
!          index the 2-particle charge-transfer  k states             !
!A charge-transfer 2-particle k-state is defined in terms of local    !
!charge transfer states as                                            !
!                                                                     !
!    |k,v;s,v'> = SUM_n exp(i*2*pi*k*n/N)|n,v;n+s,v'> / sqrt(N)       !
!    where n is the cationic molecule with v vibrational quanta       !
!    in the shifted potential well and n+s is the anionic molecule    !
!    with v' vibrational quanta in its shifted well. All other        !
!    molecules are assumed to be in their ground states               !
!*********************************************************************!'''
def index_ct(kount):
    #integer cvib, s, avib
    #allocate the 2-particle charge-transfer index and 
    #initialize as empty
    nx_ct=np.zeros(vibmax,nubnd-nlbnd,vibmax)
    #index the 2-particle charge-transfer states
    #|k,cvib;s,avib> -> nx_ct( cvib, s, avib )
    for cvib in range(0,vibmax):#vibration on cation molecule
        for s in range(nlbnd,nubnd):#!anion displacement from cation
            if s==0:#!displacement cant be zero
                break
            for avib in range(0,vibmax):#vibration on anion molecule
                if (cvib+avib>vibmax):#truncate at vibmax
                    break
                kount+=1
                nx_ct(cvib,s,avib)=kount

# Frank Condon Table

In [ ]:
'''!********************************************************************!
!     initialize the vibrational overlap tables                      !
!********************************************************************!'''
subroutine set_fctable()
    use commonvar
    implicit none

    integer vibg, vibn, vibc, viba
    real*8, external :: volap

    !allocate space for the vibrational overlap tables
    allocate( fc_gf ( 0:vibmax, 0:vibmax ) )
    allocate( fc_gc ( 0:vibmax, 0:vibmax ) )
    allocate( fc_ga ( 0:vibmax, 0:vibmax ) )
    allocate( fc_cf ( 0:vibmax, 0:vibmax ) )
    allocate( fc_af ( 0:vibmax, 0:vibmax ) )
    
    !Generate the vibrational overlap tables. Ground state potential
    !well minimum is the reference, all others are shifted by
    !lambda

    ! ground to frenkel
    do vibg = 0,vibmax
    do vibn = 0,vibmax
        fc_gf(vibg, vibn) = volap( 0.d0, vibg, lambda_n, vibn )
    enddo
    enddo

    !ground to cation
    do vibg = 0,vibmax
    do vibc = 0,vibmax
        fc_gc(vibg, vibc) = volap( 0.d0, vibg, lambda_c, vibc )
    enddo
    enddo
        
    !ground to anion
    do vibg = 0,vibmax
    do viba = 0,vibmax
        fc_ga(vibg, viba) = volap( 0.d0, vibg, lambda_a, viba )
    enddo
    enddo

    !cation to frenkel
    do vibn = 0,vibmax
    do vibc = 0,vibmax
        fc_cf(vibc, vibn) = volap( lambda_c, vibc, lambda_n, vibn )
    enddo
    enddo
    
    !anion to frenkel
    do vibn = 0,vibmax
    do viba = 0,vibmax
        fc_af(viba, vibn) = volap( lambda_a, viba, lambda_n, vibn )
    enddo
    enddo
    
end subroutine
'''!********************************************************************!
!    Return the vibrational overlap for displaced harmonic           !
!    oscillators. The lambda are proportional to the well minimum    !
!    and when squared are equivalent to the Huang-Rhys parameter.    !
!    The vibrational overlap factors are given by the formula        !
!                                                                    !
!    <m|n>=sqrt(m!n!)exp(-lambda^2/2) *                              !
!          SUM_l^(min(m,n))                                          !
!                (-1)^(m-l)/[(m-l)!l!(n-l)!] *                       !
!                lambda^(m+n-2l)                                     !
!                                                                    !
!    which can be derived from the recursion relations found         !
!    on page 167 of MODERN OPTICAL SPECTROSCOPY                      !
!    here lambda is proportional to the equilibrium displacement     !
!********************************************************************!'''
real*8 function volap( lambda1, vib1, lambda2, vib2 )
    implicit none

    integer, intent (in) :: vib1, vib2
    real*8, intent(in) :: lambda1, lambda2
    integer k
    integer, external :: factorial
    real*8 lambda
    
    !calculate the displacement between the two potential wells
    lambda = lambda2 - lambda1

    volap = 0.d0
    !calculate the vibrational overlap
    !first calculate the summation
    do k = 0, min( vib1, vib2 )
        volap = volap+(-1.d0)**(vib2-k)/        &
                (factorial(vib1-k)*factorial(k)*&
                 factorial(vib2-k))*            &
                 lambda**(vib1+vib2-2*k)
    end do
    volap = volap*dsqrt(1.d0*factorial(vib1)*   &
                             factorial(vib2))*  &
                             dexp(-1.d0*        &
                             lambda**2/2.d0)

end function
'''!********************************************************************!
!    calcuate factorial                                              !
!********************************************************************!'''
integer function factorial( n )
    implicit none

    integer, intent(in) :: n
    integer i

    if ( n < 0 ) then
        print*, 'Factorial not calculatable for ', n, ' aborting.'
        stop
    end if

    factorial = 1
    do i = 1, n
        factorial = factorial*i
    end do

end function


In [21]:
def exciton_main():
    #read the user input file and set simulation parameters
    parameters=read_in_para()
    #index the multiparticle basis set
    kount = 0
    if (parameters[one_state]):
        index_1p(kount)
    if (parameters[two_state]):
        index_2p(knount)
    if (parameters[ct_state]):
        index_ct(knount)

    #make sure the number of requested eigenstates is less than the
    #total number of eigenstates possible
    esnum = min(esnum,kount) 
    
    return parameters
    

In [23]:
exciton_main()

Reading the input file...
**********************************
**********************************
**********************************
**********************************


{'task_title': 'example',
 'nmol': 20.0,
 'vibmax': 4.0,
 'hw': '1400.0',
 'lambda_n': None,
 'lambda_c': None,
 'lambda_a': None,
 'JCoul': '700.0',
 'ES1': '14000.0',
 'te': '700.0',
 'th': '700.0',
 'ECT': '14000.0',
 'ECTInf': '20000.0',
 'one_state': True,
 'two_state': True,
 'ct_state': True,
 'abs_lw': '250.0',
 'esnum': 10000.0,
 'lambda': '1.0',
 'lambda+': '0.50',
 'lambda-': '0.50'}

In [ ]:
#program exciton_main
    #integer k   !the k-index
    
    #!read the user input file and set simulation parameters 
   
    #index the multiparticle basis set
    
    #make sure the number of requested eigenstates is less than the
    #total number of eigenstates possible

    #build the franck-condon table for the vibrational overlap factors
    call set_fctable()   
    
    #allocate space for the Hamiltonian matrix and eigenvalue array
    allocate ( h(kount,kount) ) !the Hamiltonian
    allocate ( eval(kount) )    !eigenvalues


    print'(a)', ' Will now build the Hamiltonian, the'//&
                   ' diminsion of each k-block is:'
    print'(i6)',  kount
    print*, '**********************************'//&
            '**********************************'

    !build each k-block of the Hamiltonian diagonalize,
    !and calculate the observables
    do k = nlbnd, nubnd

        !initialize hamiltonian to zero
        h = complex_zero
        
        !build the hamiltonian
        if ( one_state ) call build_h1p(k)
        if ( two_state ) call build_h2p(k)
        if ( one_state .and. two_state ) call build_h1p2p(k)
        if ( ct_state ) call build_hct(k)
        if ( one_state .and. ct_state ) call build_h1pct(k)
        if ( two_state .and. ct_state ) call build_h2pct(k)

        !diagonalize the hamiltonian
        if ( k == 0 .or. esnum == kount ) then
            call diagonalize(h, kount, eval, 'A', kount )
        else
            call diagonalize(h, kount, eval, 'I', esnum)
        end if   
        
        !calculate absorption spectrum
        !(only k=0 absorbes assuming parallel dipoles)
        if ( k == 0 ) then
            call absorption()
        end if
        call dispersion(k)
        
        print*, 'Done with wavevector k: ', k
    end do

    !write the parameter file
    call para_out()
    print*, '**********************************'//&
            '**********************************'
    print*, 'Program exited successfully.'

end program
